**S-box**

In [175]:
sbox = '63 7c 77 7b f2 6b 6f c5 30 01 67 2b fe d7 ab 76' \
       'ca 82 c9 7d fa 59 47 f0 ad d4 a2 af 9c a4 72 c0' \
       'b7 fd 93 26 36 3f f7 cc 34 a5 e5 f1 71 d8 31 15' \
       '04 c7 23 c3 18 96 05 9a 07 12 80 e2 eb 27 b2 75' \
       '09 83 2c 1a 1b 6e 5a a0 52 3b d6 b3 29 e3 2f 84' \
       '53 d1 00 ed 20 fc b1 5b 6a cb be 39 4a 4c 58 cf' \
       'd0 ef aa fb 43 4d 33 85 45 f9 02 7f 50 3c 9f a8' \
       '51 a3 40 8f 92 9d 38 f5 bc b6 da 21 10 ff f3 d2' \
       'cd 0c 13 ec 5f 97 44 17 c4 a7 7e 3d 64 5d 19 73' \
       '60 81 4f dc 22 2a 90 88 46 ee b8 14 de 5e 0b db' \
       'e0 32 3a 0a 49 06 24 5c c2 d3 ac 62 91 95 e4 79' \
       'e7 c8 37 6d 8d d5 4e a9 6c 56 f4 ea 65 7a ae 08' \
       'ba 78 25 2e 1c a6 b4 c6 e8 dd 74 1f 4b bd 8b 8a' \
       '70 3e b5 66 48 03 f6 0e 61 35 57 b9 86 c1 1d 9e' \
       'e1 f8 98 11 69 d9 8e 94 9b 1e 87 e9 ce 55 28 df' \
       '8c a1 89 0d bf e6 42 68 41 99 2d 0f b0 54 bb 16'.replace(" ", "")

sbox = bytearray.fromhex(sbox)

inv_sbox = '52 09 6a d5 30 36 a5 38 bf 40 a3 9e 81 f3 d7 fb' \
           '7c e3 39 82 9b 2f ff 87 34 8e 43 44 c4 de e9 cb' \
           '54 7b 94 32 a6 c2 23 3d ee 4c 95 0b 42 fa c3 4e' \
           '08 2e a1 66 28 d9 24 b2 76 5b a2 49 6d 8b d1 25' \
           '72 f8 f6 64 86 68 98 16 d4 a4 5c cc 5d 65 b6 92' \
           '6c 70 48 50 fd ed b9 da 5e 15 46 57 a7 8d 9d 84' \
           '90 d8 ab 00 8c bc d3 0a f7 e4 58 05 b8 b3 45 06' \
           'd0 2c 1e 8f ca 3f 0f 02 c1 af bd 03 01 13 8a 6b' \
           '3a 91 11 41 4f 67 dc ea 97 f2 cf ce f0 b4 e6 73' \
           '96 ac 74 22 e7 ad 35 85 e2 f9 37 e8 1c 75 df 6e' \
           '47 f1 1a 71 1d 29 c5 89 6f b7 62 0e aa 18 be 1b' \
           'fc 56 3e 4b c6 d2 79 20 9a db c0 fe 78 cd 5a f4' \
           '1f dd a8 33 88 07 c7 31 b1 12 10 59 27 80 ec 5f' \
           '60 51 7f a9 19 b5 4a 0d 2d e5 7a 9f 93 c9 9c ef' \
           'a0 e0 3b 4d ae 2a f5 b0 c8 eb bb 3c 83 53 99 61' \
           '17 2b 04 7e ba 77 d6 26 e1 69 14 63 55 21 0c 7d'.replace(" ", "")

inv_sbox = bytearray.fromhex(inv_sbox)

**Hàm phụ**

In [176]:
# Chuyển từ byte sang state
def byte2state(text):
    state = []
    for i in range(len(text) // 4):
        state.append(text[i*4:(i+1)*4])
    return state

# Chuyển từ state sang byte
def state2byte(state):
    return bytes(state[0] + state[1] + state[2] + state[3])

# xor hai bytes
def xor_bytes(a, b):
    return bytes([x ^ y for (x, y) in zip(a, b)])

def xtime(a):
    if a & 0x80:
        return ((a << 1) ^ 0x1b) & 0xff
    return a << 1

def xtimes_0e(b):
    # 0x0e = 14 = b1110 = ((x * 2 + x) * 2 + x) * 2
    return xtime(xtime(xtime(b) ^ b) ^ b)


def xtimes_0b(b):
    # 0x0b = 11 = b1011 = ((x*2)*2+x)*2+x
    return xtime(xtime(xtime(b)) ^ b) ^ b


def xtimes_0d(b):
    # 0x0d = 13 = b1101 = ((x*2+x)*2)*2+x
    return xtime(xtime(xtime(b) ^ b)) ^ b


def xtimes_09(b):
    # 0x09 = 9  = b1001 = ((x*2)*2)*2+x
    return xtime(xtime(xtime(b))) ^ b

**Operations**

In [177]:
def AddRoundKey(state, roundkey):
    for r in range(len(state)):
        state[r] = [state[r][c] ^ roundkey[r][c] for c in range(len(state[0]))]
    return state

def SubBytes(state):
    for r in range(len(state)):
        state[r] = [sbox[state[r][c]] for c in range(len(state[0]))]
    return state

def ShiftRows(state):
    state[0][1], state[1][1], state[2][1], state[3][1] = state[1][1], state[2][1], state[3][1], state[0][1]
    state[0][2], state[1][2], state[2][2], state[3][2] = state[2][2], state[3][2], state[0][2], state[1][2]
    state[0][3], state[1][3], state[2][3], state[3][3] = state[3][3], state[0][3], state[1][3], state[2][3]
    return state

def MixSingleColumn(col):
    c_0 = col[0]
    all_xor = col[0] ^ col[1] ^ col[2] ^ col[3]
    col[0] ^= all_xor ^ xtime(col[0] ^ col[1])
    col[1] ^= all_xor ^ xtime(col[1] ^ col[2])
    col[2] ^= all_xor ^ xtime(col[2] ^ col[3])
    col[3] ^= all_xor ^ xtime(c_0 ^ col[3])
    return col

def MixColumns(state):
    for i in range(len(state)):
        state[i] = MixSingleColumn(state[i])
    return state



# Các quá trình nghịch đảo
def inv_ShiftRows(state):
    state[1][1], state[2][1], state[3][1], state[0][1] = state[0][1], state[1][1], state[2][1], state[3][1]
    state[2][2], state[3][2], state[0][2], state[1][2] = state[0][2], state[1][2], state[2][2], state[3][2]
    state[3][3], state[0][3], state[1][3], state[2][3] = state[0][3], state[1][3], state[2][3], state[3][3]
    return state

def inv_SubBytes(state):
    for r in range(len(state)):
        state[r] = [inv_sbox[state[r][c]] for c in range(len(state[0]))]
    return state

def inv_MixSingleColumn(col):
    c_0, c_1, c_2, c_3 = col[0], col[1], col[2], col[3]
    col[0] = xtimes_0e(c_0) ^ xtimes_0b(c_1) ^ xtimes_0d(c_2) ^ xtimes_09(c_3)
    col[1] = xtimes_09(c_0) ^ xtimes_0e(c_1) ^ xtimes_0b(c_2) ^ xtimes_0d(c_3)
    col[2] = xtimes_0d(c_0) ^ xtimes_09(c_1) ^ xtimes_0e(c_2) ^ xtimes_0b(c_3)
    col[3] = xtimes_0b(c_0) ^ xtimes_0d(c_1) ^ xtimes_09(c_2) ^ xtimes_0e(c_3)
    
    return col


def inv_MixColumns(state):
    for i in range(len(state)):
        state[i] = inv_MixSingleColumn(state[i])
    return state

In [178]:
def RotWord(word):
    return word[1:] + word[:1]

def SubWord(word):
    substituted_word = bytes(sbox[i] for i in word)
    return substituted_word

def rcon(i):
    rcon_lookup = bytearray.fromhex('01020408102040801b36')
    rcon_value = bytes([rcon_lookup[i-1], 0, 0, 0])
    return rcon_value

def KeyExpansion(key):
    # [w0, w1, w2, w3] chính là state của key 
    w = byte2state(key)
    
    # tính các w còn lại
    for i in range(4, 44):
        temp = w[i-1]
        if i % 4 == 0:
            temp = xor_bytes(SubWord(RotWord(temp)), rcon(i // 4))
        w.append(xor_bytes(w[i - 4], temp))
    # w là list chuyển
    return [w[i*4:(i+1)*4] for i in range(len(w) // 4)]

In [179]:
# Hàm lập mã thực hiện với key có độ dài 128 bit
def Encrypt(plainText, key):
    # Khởi tạo ma trận trạng thái
    state = byte2state(plainText)

    # Quá trình mở rộng khóa
    keySchedule = KeyExpansion(key)        
    
    # lặp
    state = AddRoundKey(state, keySchedule[0])
    print('Round 0:', state2byte(state).hex(), state2byte(keySchedule[0]).hex())
    for round in range(1, 10):
        state = SubBytes(state)
        state = ShiftRows(state)
        state = MixColumns(state)
        state = AddRoundKey(state, keySchedule[round])
        print('Round', round, ':', state2byte(state).hex(), state2byte(keySchedule[round]).hex())

    state = SubBytes(state)
    state = ShiftRows(state)
    state = AddRoundKey(state, keySchedule[-1])
    print('Round 10:', state2byte(state).hex(), state2byte(keySchedule[-1]).hex())
    
    # Chuyển từ ma trận sang dạng byte
    cipherText = state2byte(state)
    return cipherText

def Decrypt(cipherText, key):
    # Khởi tạo ma trận trạng thái
    state = byte2state(cipherText)

    # Quá trình mở rộng khóa
    keySchedule = KeyExpansion(key)[::-1]
    
    # lặp
    state = AddRoundKey(state, keySchedule[0])
    print('Round 0:', state2byte(state).hex(), state2byte(keySchedule[0]).hex())
    for round in range(1, 10):
        state = inv_ShiftRows(state)
        state = inv_SubBytes(state)
        state = AddRoundKey(state, keySchedule[round])
        state = inv_MixColumns(state)
        print('Round', round, ':', state2byte(state).hex(), state2byte(keySchedule[round]).hex())

    state = inv_ShiftRows(state)
    state = inv_SubBytes(state)
    state = AddRoundKey(state, keySchedule[-1])
    print('Round 10:', state2byte(state).hex(), state2byte(keySchedule[-1]).hex())
    
    # Chuyển từ ma trận sang dạng byte
    cipherText = state2byte(state)
    return cipherText

In [180]:
plainText = '54776F204F6E65204E696E652054776F'
key = '5468617473206D79204B756E67204675'
plainText = bytearray.fromhex(plainText)
key = bytearray.fromhex(key)

print('Encrypt:')
cipherText = Encrypt(plainText, key)
print('\nCipher Text:', cipherText.hex())

print('\nDecrypt:')
text = Decrypt(cipherText, key)
print('\nRecovered Text:', text.hex())

Encrypt:
Round 0: 001f0e543c4e08596e221b0b4774311a 5468617473206d79204b756e67204675
Round 1 : 5847088b15b61cba59d4e2e8cd39dfce e232fcf191129188b159e4e6d679a293
Round 2 : 43c6a9620e57c0c80908ebfe3df87f37 56082007c71ab18f76435569a03af7fa
Round 3 : 7876305470767d23993c375b4b3934f1 d2600de7157abc686339e901c3031efb
Round 4 : b1ca51ed08fc54e104b1c9d3e7b26c20 a11202c9b468bea1d75157a01452495b
Round 5 : 9b512068235f22f05d1cbd322f389156 b1293b3305418592d210d232c6429b69
Round 6 : 149325778fa42be8c06024405e0f9275 bd3dc287b87c47156a6c9527ac2e0e4e
Round 7 : 53398e5d430693f84f0a3b95855257bd cc96ed1674eaaa031e863f24b2a8316a
Round 8 : 66253c7470ce5aa8afd30f0aa3731354 8e51ef21fabb4522e43d7a0656954b6c
Round 9 : 09668b78a2d19a65f0fce6c47b3b3089 bfe2bf904559fab2a16480b4f7f1cbd8
Round 10: 29c3505f571420f6402299b31a02d73a 28fddef86da4244accc0a4fe3b316f26

Cipher Text: 29c3505f571420f6402299b31a02d73a

Decrypt:
Round 0: 013e8ea73ab004bc8ce23d4d2133b81c 28fddef86da4244accc0a4fe3b316f26
Round 1 : 338b762051667d